In [1]:
import os
import textwrap
import json
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


import cx_Oracle

from datetime import datetime

import PIL.Image

c:\Users\sean.chang\AppData\Local\anaconda3\envs\bibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# the path of input 
dirpath = r"\\10.25.25.216\FidelisFileServer\04數據與分析部\部門資料\LLM資料\華紙-營運績效分析月報\masked"

topic_name = "華紙-營運績效分析月報"

prompt = "請用中文說明報表包含哪些指標分析內容"

### Helper functions

In [2]:
def read_json(data_path):
    with open(data_path) as f:
        data = json.load(f)
    return data

In [3]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Generate description for dashboard

In [4]:
info_path = '.env/info.json'
info = read_json(info_path)
API_KEY= info['gkey']
genai.configure(api_key=API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')


In [5]:
oracle_client_dir = './files/oracle/'

oracle_client_path = os.path.join(oracle_client_dir, os.listdir(oracle_client_dir)[0])

cx_Oracle.init_oracle_client(lib_dir=oracle_client_path)

In [6]:
db_info  = info['dw_conn']
user = db_info['user']
pw = db_info['pw']
IP = db_info['IP']
port = db_info['port']
service_name = db_info['service_name']

# datetime object containing current date and time
now = datetime.now()

# 連線到 BI DWH 資料庫 oracle
dsn_tns = cx_Oracle.makedsn(IP, port, service_name=service_name)
DWH_conn = cx_Oracle.connect(user=user, password=pw, dsn=dsn_tns)

# 建立 cursor
DWH_cursor = DWH_conn.cursor()


In [15]:
def _insert_data(table_name, topic_name,topic_pic_name,topic_content,now ):
    sqlstat = "insert into " + table_name + " ( topic_name, topic_pic_name, topic_content, creation_date) values ( :topic_name,:topic_pic_name,:topic_content, :now )"
    try:
        # establish a new connection
        dsn_tns = cx_Oracle.makedsn('10.25.66.64', '1521', service_name='biprod')
        DWH_conn = cx_Oracle.connect(user='YFYDW', password='!(Yfy123456)', dsn=dsn_tns)
        with DWH_conn as connection:
            # create a cursor
            with connection.cursor() as cursor:
                # execute the insert statement
                cursor.execute(sqlstat, [topic_name, topic_pic_name, topic_content, now])
                # commit work
                connection.commit()
    except cx_Oracle.Error as error:
            print('Error occurred:')
            print(error)

In [ ]:
# dirpath = r"\\10.25.25.216\FidelisFileServer\04數據與分析部\部門資料\LLM資料\華紙-營運績效分析月報\masked"
d1 = os.listdir(dirpath)
table_name = "ai_report"
for i  in d1:
    print(i)
    if i != "Thumbs.db" :
        filename = dirpath + "\\" + i
        img = PIL.Image.open(filename)
        response = model.generate_content([prompt, img], stream=True)
        response.resolve()
        topic_content = response.text
        topic_pic_name = filename
        print(topic_content)
        _insert_data( table_name,topic_name,topic_pic_name,topic_content,now)